Load in required libraries, below.

In [23]:
# data 
import numpy as np
from sklearn.model_selection import train_test_split


## SageMaker Resources

The below cell stores the SageMaker session and role (for creating estimators and models), and creates a default S3 bucket. After creating this bucket, locally stored data can be uploaded to S3.

In [24]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

In [25]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()
prefix='cnn-wendy-data-2b'
prefix_output='cnn-wendy-model-2b'

Here we retrieve the dataset of images and we upload it to S3


In [4]:
#!wget -nc https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data_2b.zip
#!unzip -qq -n wendy_cnn_frames_data_2b.zip -d wendy_cnn_frames_data_2b 

--2020-10-25 23:18:01--  https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data_2b.zip
Resolving da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)... 52.219.75.100
Connecting to da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)|52.219.75.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1094263352 (1.0G) [application/zip]
Saving to: ‘wendy_cnn_frames_data_2b.zip’

wendy_cnn_frames_da 100%[===================>]   1.02G  94.6MB/s    in 11s     

2020-10-25 23:18:12 (94.9 MB/s) - ‘wendy_cnn_frames_data_2b.zip’ saved [1094263352/1094263352]



In [6]:

# upload to S3. Skip if already uploaded. This can take a while.
#print('Uploading data to {}'.format(prefix))
#input_data = sagemaker_session.upload_data(path='wendy_cnn_frames_data_2b', bucket=bucket, key_prefix=prefix)
#print('Data uploaded to {}'.format(input_data))

Uploading data to cnn-wendy-data-2b
Data uploaded to s3://sagemaker-eu-central-1-283211002347/cnn-wendy-data-2b


In [4]:
# location to input data can be written down here, if known
input_data='s3://sagemaker-eu-central-1-283211002347/cnn-wendy-data-2b'

After uploading images to S3, we can define and train the estimator


In [5]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path

output_path = 's3://{}/{}'.format(bucket, prefix_output)
print('Output path for models is {}'.format(output_path))

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='letsplay_classifier',
                    role=role,
                    framework_version='1.6',
                    train_instance_count=1,
                    train_instance_type='ml.p2.8xlarge',
                    train_volume_size = 10,
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'img-width': 320,
                        'img-height': 180,
                        'batch-size': 16,
                        'layer-cfg': 'B',
                        'epochs': 5
                    })

Output path for models is s3://sagemaker-eu-central-1-283211002347/cnn-wendy-model-2b


## Train the Estimator

After instantiating the estimator, we train it with a call to `.fit()`. 

In [6]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-26 00:49:31 Starting - Starting the training job...
2020-10-26 00:49:33 Starting - Launching requested ML instances.........
2020-10-26 00:51:28 Starting - Preparing the instances for training.........
2020-10-26 00:52:50 Downloading - Downloading input data........................
2020-10-26 00:56:51 Training - Downloading the training image.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-26 00:57:07,276 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-10-26 00:57:07,352 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-26 00:57:07,361 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-26 00:57:07,839 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requirements.txt
2020-10-26 00:57:09,886 sagemaker-training

In [7]:
print(estimator.model_data)
model_data = estimator.model_data
model_data = 's3://sagemaker-eu-central-1-283211002347/cnn-wendy-model-2b/pytorch-training-2020-10-26-00-49-31-414/output/model.tar.gz'

s3://sagemaker-eu-central-1-283211002347/cnn-wendy-model-2b/pytorch-training-2020-10-26-00-49-31-414/output/model.tar.gz


In [18]:
model_data = 's3://sagemaker-eu-central-1-283211002347/cnn-wendy-model-2b/pytorch-training-2020-10-26-00-49-31-414/output/model.tar.gz'

We set up a model that can predict the class of an image

### Deploy the trained model

We deploy our model to create a predictor. We'll use this to make predictions on our data and evaluate the model.

In [19]:
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=model_data,
                     role = role,
                     framework_version='1.6',
                     entry_point='predict.py',
                     source_dir='letsplay_classifier')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [20]:
%%time
# deploy and create a predictor
              
predictor = model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')


'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------------!CPU times: user 38.4 s, sys: 5.17 s, total: 43.6 s
Wall time: 10min 13s


In [21]:
# the endpoint where the predictor is located
endpoint_name = predictor.endpoint
print(endpoint_name)

pytorch-inference-2020-10-26-04-51-38-837


Now that the model is deployed, we check how the predictor performs on our full dataset,
ensuring that the predictions make sense. We produce a classification report.


In [1]:

endpoint_name='pytorch-inference-2020-10-26-04-51-38-837'

In [12]:

from sklearn.metrics import classification_report
from letsplay_classifier.endpoint import evaluate
y_true, y_pred = evaluate(endpoint_name, 'wendy_cnn_frames_data_2b', 0.15)


5000 processed up to 16653
10000 processed up to 33371


In [13]:
report = classification_report(y_true=y_true, y_pred=y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       953
           1       0.98      0.99      0.98       158
           2       0.99      0.99      0.99      4713
           3       1.00      0.89      0.94        27
           4       0.98      0.98      0.98      1033

    accuracy                           0.99      6884
   macro avg       0.98      0.97      0.98      6884
weighted avg       0.99      0.99      0.99      6884



In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[ 947,    0,    3,    0,    3],
       [   1,  157,    0,    0,    0],
       [  30,    2, 4666,    0,   15],
       [   1,    2,    0,   24,    0],
       [   0,    0,   22,    0, 1011]])

In [4]:
!wget -nc https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_E67.zip
!unzip -qq -n wendy_cnn_frames_E67.zip -d wendy_cnn_frames_E67

--2020-10-25 15:59:24--  https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_E67.zip
Resolving da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)... 52.219.75.88
Connecting to da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)|52.219.75.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76964860 (73M) [application/zip]
Saving to: ‘wendy_cnn_frames_E67.zip’

wendy_cnn_frames_E6 100%[===================>]  73.40M  97.8MB/s    in 0.8s    

2020-10-25 15:59:25 (97.8 MB/s) - ‘wendy_cnn_frames_E67.zip’ saved [76964860/76964860]



In [2]:
from letsplay_classifier.interval.predict_intervals_endpoint import evaluate
evaluate(endpoint_name, 'wendy_cnn_frames_E67/E67', class_names= ['Battle', 'Hideout', 'Other', 'Siege', 'Tournament'])

23:04-28:04 | Battle : 11% , Hideout : 43% , Other : 7% , Siege : 36% 
42:48-42:52 | Other : 35% , Tournament : 65% 
43:44-43:46 | ????? 
46:24-47:44 | Battle : 38% , Other : 12% , Tournament : 49% 
51:00-51:04 | Other : 30% , Tournament : 70% 
52:30-53:02 | Battle : 56% , Hideout : 30% , Other : 12% 
54:36-56:00 | Battle : 57% , Other : 11% , Tournament : 31% 
01:03:52-01:05:42 | Battle : 62% , Other : 10% , Tournament : 26% 
01:14:00-01:16:36 | Battle : 19% , Other : 7% , Tournament : 72% 
01:17:50-01:19:16 | Battle : 30% , Other : 9% , Tournament : 59% 
01:33:12-01:34:22 | Battle : 94% 
01:38:16-01:43:50 | Battle : 74% , Other : 8% , Tournament : 17% 
01:43:54-01:46:06 | Battle : 72% , Other : 10% , Tournament : 16% 
01:49:00-01:50:38 | Battle : 94% 
01:50:48-01:53:32 | Battle : 90% , Other : 8% 
01:55:52-01:57:46 | Battle : 92% 


## Delete the Endpoint

Finally, I've added a convenience function to delete prediction endpoints after we're done with them. 

In [3]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=endpoint_name)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [4]:
# delete the predictor endpoint 
delete_endpoint(predictor)

NameError: name 'predictor' is not defined